# Évaluer sur l'ensemble de test (10 %) 

In [61]:
from comet_ml import API
import joblib

In [62]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

### Loading models

In [63]:
load_dotenv(r".env")
comet_api_key = os.environ.get('COMET_API_KEY')

In [64]:
api = API(api_key=comet_api_key)

#get the Model object
model = api.get_model(workspace="ift6758-a02", model_name='best-xgboost-model')

#Download a Registry Model:
model.download("2.0.0", expand=True)

In [65]:
loaded_model = joblib.load('None/best_XGBoost_selected_features_xgb.json')

### Loading data

In [66]:
data_path = os.environ.get('DATA_DIR')

In [67]:
test_data_path = data_path+"\\nhl_data_2020_previous_v2_0.csv"
test_data=pd.read_csv(test_data_path)

In [68]:
test_data.sample(3)

,period,periodTimeInSeconds,isGoal,typeDeTir,x,y,distanceToNet,relativeAngleToNet,previousEventTypeId,previousX,previousY,distanceFromPrevious,timeDiff,rebond,angleChange,vitesse,season
28597,2,1032,0,Wrist Shot,-41.0,13.0,50.695167,14.858614,FACEOFF,-69.0,22.0,29.41,23,False,0.0,1.278696,2020
31637,2,161,0,Wrist Shot,75.0,-17.0,22.671568,-5.882446,MISSED_SHOT,63.0,9.0,28.64,5,False,0.0,5.728000,2020
48002,1,1017,0,Tip-In,77.0,1.0,13.038405,0.343084,MISSED_SHOT,65.0,14.0,17.69,5,False,0.0,3.538000,2020


In [69]:
test_data.dropna(inplace=True)
test_data = test_data.replace([np.inf, -np.inf], np.nan).dropna()

In [70]:
dummy_object = pd.get_dummies(test_data[['typeDeTir', 'previousEventTypeId']])
data = test_data.merge(dummy_object, left_index=True, right_index=True)
test_data_fin = data.drop(labels = ['typeDeTir', 'previousEventTypeId'], axis = 1)

In [71]:
selected_features = ['period', 'periodTimeInSeconds', 'y', 'distanceToNet', 'timeDiff', 'rebond', 'vitesse', 'typeDeTir_Backhand', 'typeDeTir_Deflected', 'typeDeTir_Slap Shot', 'typeDeTir_Snap Shot', 'typeDeTir_Tip-In', 'typeDeTir_Wrap-around', 'typeDeTir_Wrist Shot', 'previousEventTypeId_FACEOFF', 'previousEventTypeId_GIVEAWAY', 'previousEventTypeId_HIT', 'previousEventTypeId_MISSED_SHOT']

In [74]:
X_test = test_data_fin.loc[:,['distanceToNet', 'relativeAngleToNet']]
y_test = test_data_fin['isGoal']

In [75]:
y_pred = loaded_model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])